**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://manipulation.csail.mit.edu/clutter.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display
from ipywidgets import Textarea

from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator
)

from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback

from manipulation.utils import FindResource

# Falling things (in 2D)

TODO(russt): Switch to planar scenegraph visualizer and an animation

In [ ]:
def clutter_gen(show_collision_geometry=False):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform([0, 0, -5])
    mu = 1.0
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])
    planar_joint_frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))

    parser = Parser(plant)

    if show_collision_geometry:  # Show collision geometry
        sdf = FindResource("models/061_foam_brick_collision_as_visual.sdf")
    else:
        sdf = FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf")
    link_name = "base_link"

    for i in range(20 if running_as_notebook else 2):
        instance = parser.AddModelFromFile(sdf, f"object{i}")
        plant.AddJoint(PlanarJoint(f"joint{i}", planar_joint_frame, plant.GetFrameByName(link_name, instance), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph,
#        delete_prefix_on_load = False,
        zmq_url=zmq_url,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-0.6, xmax=.6, ymin=-.1, ymax=0.5)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    plant_context = plant.GetMyContextFromRoot(simulator.get_mutable_context())

    rs = np.random.RandomState()
    z = 0.1
    for i in range(1, plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        joint.set_pose(plant_context, [rs.uniform(-.5, .5), z], rs.uniform(-np.pi/2.0, np.pi/2.0))
        z += 0.2

#    simulator.set_target_realtime_rate(0.05)
    simulator.AdvanceTo(4.0 if running_as_notebook else 0.1)
    
clutter_gen()

# Falling things (in 3D)

TODO(russt): Switch to no visualization, just add a camera to display the final scene.

In [ ]:
def clutter_gen():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform([0, 0, -5])
    mu = 1.0
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])
    planar_joint_frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))

    parser = Parser(plant)

    ycb = [("cracker", "003_cracker_box.sdf"), 
           ("sugar", "004_sugar_box.sdf"), 
           ("soup", "005_tomato_soup_can.sdf"), 
           ("mustard", "006_mustard_bottle.sdf"), 
           ("gelatin", "009_gelatin_box.sdf"), 
           ("meat", "010_potted_meat_can.sdf")]

    rs = np.random.RandomState()
    for i in range(20 if running_as_notebook else 2):
        object_num = rs.randint(len(ycb))
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        link_name = "base_link_" + ycb[object_num][0]
        instance = parser.AddModelFromFile(sdf, f"object{i}")
        plant.AddJoint(PlanarJoint(f"joint{i}", planar_joint_frame, plant.GetFrameByName(link_name, instance), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph,
#        delete_prefix_on_load = False,
        zmq_url=zmq_url,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-0.6, xmax=.6, ymin=-.1, ymax=0.5)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    plant_context = plant.GetMyContextFromRoot(simulator.get_mutable_context())

    z = 0.1
    for i in range(1, plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        joint.set_pose(plant_context, [rs.uniform(-.5, .5), z], rs.uniform(-np.pi/2.0, np.pi/2.0))
        z += 0.2

#    simulator.set_target_realtime_rate(0.05)
    simulator.AdvanceTo(4.0 if running_as_notebook else 0.1)
    
clutter_gen()

# Contact force "inspector"

In [ ]:
def contact_force_inspector(slope=0.0, mu=1.0, show_collision_geometry=False, second_brick=False):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform(RotationMatrix.MakeYRotation(slope), [0, 0, -5.05])
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])

    parser = Parser(plant)
    if show_collision_geometry:
        brick_sdf = FindResource("models/061_foam_brick_collision_as_visual.sdf")
    else:
        brick_sdf = FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf")
    parser.AddModelFromFile(brick_sdf)
    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    plant.AddJoint(PlanarJoint("brick", frame, plant.GetFrameByName("base_link"), damping=[0,0,0]))

    if second_brick:
        brick2 = parser.AddModelFromFile(brick_sdf, "brick2")
        plant.AddJoint(PlanarJoint("brick2", frame, plant.GetFrameByName("base_link", brick2), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-.2, xmax=.2, ymin=-.2, ymax=0.3)

    contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
                                           force_threshold=0.0,
                                           contact_force_scale=1.0, 
                                           contact_force_radius=0.002,   
                                           plant=plant))
    builder.Connect(scene_graph.get_pose_bundle_output_port(),
                    contact_visualizer.GetInputPort("pose_bundle"))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    diagram = builder.Build()
    meshcat.load()
    context = diagram.CreateDefaultContext()

    textoutput = Textarea(value="", description="contact info: ", layout={'width':'800px','height':'100px'}, style={'description_width':'initial'})
    display(textoutput)
    def my_callback(plant_context):
        results = plant.get_contact_results_output_port().Eval(plant_context)
        textoutput.value = "" if results.num_point_pair_contacts()>0 else "no contact"
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            textoutput.value += f"slip speed:{info.slip_speed():.4f}, depth:{pair.depth:.4f}, "
            textoutput.value += "force:" + np.array2string(info.contact_force(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"

    lower_limit = [-0.1, -0.1, -np.pi/2.0]
    upper_limit = [0.1, 0.1, np.pi/2.0]
    if second_brick:
        lower_limit += lower_limit
        upper_limit += upper_limit
        plant.SetPositions(
            plant.GetMyContextFromRoot(context),
            [0, 0, 0, 0.07, 0.07, 0.0])

    MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback, resolution=0.001,
        lower_limit=lower_limit, upper_limit=upper_limit)    

contact_force_inspector(
    slope=0.1, 
    mu=0.5,
    show_collision_geometry=False,
    second_brick=True)